In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [ ]:
img_height, img_width = 224, 224 
batch_size = 32


In [4]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [5]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [6]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [7]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


2025-04-02 11:19:27.822491: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-02 11:19:27.822523: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-02 11:19:27.822534: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-02 11:19:27.822734: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 11:19:27.822752: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [9]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
)

Epoch 1/100


2025-04-02 11:19:29.631168: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.6665 - accuracy: 0.6060

2025-04-02 11:19:38.269030: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 11s 375ms/step - loss: 0.6665 - accuracy: 0.6060 - val_loss: 0.5733 - val_accuracy: 0.7349
Epoch 2/100
21/21 [==============================] - 7s 341ms/step - loss: 0.6071 - accuracy: 0.7308 - val_loss: 0.5758 - val_accuracy: 0.7349
Epoch 3/100
21/21 [==============================] - 7s 337ms/step - loss: 0.5950 - accuracy: 0.7338 - val_loss: 0.5576 - val_accuracy: 0.7349
Epoch 4/100
21/21 [==============================] - 7s 340ms/step - loss: 0.5786 - accuracy: 0.7308 - val_loss: 0.5504 - val_accuracy: 0.7349
Epoch 5/100
21/21 [==============================] - 7s 335ms/step - loss: 0.5576 - accuracy: 0.7398 - val_loss: 0.5477 - val_accuracy: 0.7349
Epoch 6/100
21/21 [==============================] - 7s 339ms/step - loss: 0.5563 - accuracy: 0.7353 - val_loss: 0.5334 - val_accuracy: 0.7349
Epoch 7/100
21/21 [==============================] - 8s 352ms/step - loss: 0.5425 - accuracy: 0.7383 - val_loss: 0.5302 - val_accuracy: 0.7349
Epoch 8/10

In [14]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

6/6 [==============================] - 2s 209ms/step - loss: 0.4330 - accuracy: 0.8193
Validation Accuracy: 81.93%
6/6 [==============================] - 1s 244ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.94      0.83       122
           1       0.36      0.09      0.15        44

    accuracy                           0.72       166
   macro avg       0.55      0.52      0.49       166
weighted avg       0.64      0.72      0.65       166


Confusion Matrix:
[[115   7]
 [ 40   4]]
